# Calculating travel time to ports and border crossings in Georgia

We have a map of a series of road improvements in Georgia, and we need to determine how these improve access to border crossings and ports

In [1]:
import sys, os, json
import rasterio, overturemaps

import pandas as pd
import geopandas as gpd
import skimage.graph as graph

from shapely.geometry import Point

from space2stats_client import Space2StatsClient

sys.path.insert(0, r"C:\WBG\Work\Code\GOSTrocks\src")
import GOSTrocks.rasterMisc as rMisc
import GOSTrocks.dataMisc as dMisc
import GOSTrocks.ntlMisc as ntlMisc
from GOSTrocks.misc import tPrint

sys.path.append(r"C:\WBG\Work\Code\GOSTnetsraster\src")
import GOSTnetsraster.market_access as ma
import GOSTnetsraster.conversion_tables as speed_tables

s2s_client = Space2StatsClient(verify_ssl=False)

%load_ext autoreload
%autoreload 2

GDAL is not installed - OGR functionality not available


In [2]:
# Input parameters
m_crs = "ESRI:54009" # Need to project data to a metres-based projection

# Define input data
base_folder = "C:/WBG/Work/Projects/GEO_Road_Improvements"
landcover_file = os.path.join(base_folder, "DATA", 'ESA_Globcover.tif')
# These are the digitized road segements that have been improved
road_segments_file = os.path.join(base_folder, "DATA", "impacted_osm_roads.gpkg")
transport_network = os.path.join(base_folder, "DATA", "Overture", "transport_network.gpkg")
major_roads_file = os.path.join(base_folder, "DATA", "Overture", "major_roads.gpkg")
major_roads_updated_file = os.path.join(base_folder, "DATA", "major_roads_final.shp")
border_crossings_file = os.path.join(base_folder, "DATA", "Border_Crossings.shp")

# WorldPop 2020 constrained, projected to m_crs
pop_file = os.path.join(base_folder, "DATA", "geo_pop_2025_CN_100m_R2025A_v1.tif")

# https://datacatalog.worldbank.org/int/search/dataset/0038118/Global---International-Ports
port_file = os.path.join(base_folder, "DATA", "GEO_ports.gpkg")

# administrative bounadaries are used to summarize population
''' # Stupid SSL errors
adm2 = s2s_client.fetch_admin_boundaries("GEO", 'ADM2')
adm1 = s2s_client.fetch_admin_boundaries("GEO", 'ADM1')
'''
global_adm2_file =  r"C:\WBG\Work\data\ADMIN\NEW_WB_BOUNDS\FOR_PUBLICATION\geojson\WB_GAD_ADM2.geojson"
adm2 = gpd.read_file(global_adm2_file)
adm2 = adm2.loc[adm2['ISO_A3'] == 'GEO']

# Define output files
friction_folder = os.path.join(base_folder, "DATA", "FRICTION")
results_folder = os.path.join(base_folder, "RESULTS")
overture_folder = os.path.join(base_folder, "DATA", "Overture")
for cFolder in [friction_folder, results_folder, overture_folder]:
    if not os.path.exists(cFolder):
        os.makedirs(cFolder)    

pre_friction_file = os.path.join(friction_folder, 'FRICTION_pre_intervention.tif')
post_friction_file = os.path.join(friction_folder, 'FRICTION_post_intervention.tif')

# Read in data
dests = gpd.read_file(port_file).to_crs(m_crs)
if not os.path.exists(landcover_file):
    global_landcover = r"R:\GLOBAL\LCVR\Globcover\2015\ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7.tif"
    in_lc = rasterio.open(global_landcover)
    temp_landcover_file = landcover_file.replace(".tif", "_temp.tif")
    local_lc = rMisc.clipRaster(in_lc, adm2, temp_landcover_file)
    temp_lc = rasterio.open(temp_landcover_file)
    proj_res = rMisc.project_raster(temp_lc, m_crs)
    with rasterio.open(landcover_file, 'w', **proj_res[1]) as outR:
        outR.write(proj_res[0])

in_lc = rasterio.open(landcover_file)
in_pop = rasterio.open(pop_file)
if in_pop.crs != in_lc.crs:
    proj_res = rMisc.standardizeInputRasters(in_pop, in_lc, pop_file.replace(".tif", "_proj.tif"))

In [3]:
# Downlaod worldcover data
tiles_geojson = r"C:\WBG\Work\data\LCVR\esa_worldcover_grid.geojson"
in_tiles = gpd.read_file(tiles_geojson)
sel_tiles = in_tiles.loc[in_tiles.intersects(adm2.union_all())]

tile_path = "s3://esa-worldcover/v200/2021/map/ESA_WorldCover_10m_2021_v200_{tile}_Map.tif"
out_folder = os.path.join(base_folder, "DATA", "WorldCover")
for idx, row in sel_tiles.iterrows():
    cur_tile_path = tile_path.format(tile=row['ll_tile'])
    cur_out = os.path.join(out_folder, f"WorldCover_{row['ll_tile']}.tif")
    if not os.path.exists(cur_out):
        command = f"aws s3 --no-sign-request --no-verify-ssl cp {cur_tile_path} {cur_out}"
        print(command)

aws s3 --no-sign-request --no-verify-ssl cp s3://esa-worldcover/v200/2021/map/ESA_WorldCover_10m_2021_v200_N39E045_Map.tif C:/WBG/Work/Projects/GEO_Road_Improvements\DATA\WorldCover\WorldCover_N39E045.tif


In [4]:
# Download roads from Overture
#Download transport network
if not os.path.exists(transport_network):    
    bbox = adm2.total_bounds.tolist()  # minx, miny, maxx, maxy
    transport = overturemaps.record_batch_reader("segment", bbox).read_all()
    transport_df = gpd.GeoDataFrame.from_arrow(transport)
    transport_df.crs = 4326
    transport_df.loc[:, ["id", "class", "subtype", "road_surface", "speed_limits", "width_rules", "geometry"]].to_file(transport_network, driver="GPKG")

# process transport to a) remove roads outside IRAQ and b) remove all roads of OSMLR class 3 and 4
if not os.path.exists(major_roads_file):
    roads = gpd.read_file(transport_network)
    roads['OSMLR_class'] = roads['class'].map(speed_tables.OSMLR_Classes)
    roads_joined = gpd.sjoin(roads, adm2, how="inner", predicate="intersects")
    major_roads = roads_joined.loc[roads_joined['OSMLR_class'].isin(['OSMLR level 1', 'OSMLR level 2']), roads.columns]
    major_roads.to_file(major_roads_file, driver="GPKG", index=False)

# After downloading the major roads, some manual edits were made to the major roads shapefile to add missing segments and update road classes.
major_roads = gpd.read_file(major_roads_updated_file)
major_roads['OSMLR_clas'].fillna('OSMLR level 1', inplace=True)
major_roads['class'].fillna('motorway', inplace=True)

C:\Users\WB411133\AppData\Local\Temp\ipykernel_12800\3899424267.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  major_roads['OSMLR_clas'].fillna('OSMLR level 1', inplace=True)
C:\Users\WB411133\AppData\Local\Temp\ipykernel_12800\3899424267.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beha

In [5]:
# Process roads to create pre and post friction surfaces
major_roads['speed'] = major_roads['class'].map(speed_tables.osm_speed_dict)
major_roads['speed'] = major_roads['speed'].fillna(10.0)
major_roads['new_speed'] = major_roads['speed'].values
# For all the roads with a Status value, increase new_speed by 20%
major_roads.loc[~major_roads['Status'].isna(), 'new_speed'] = major_roads.loc[~major_roads['Status'].isna(), 'speed'] * 1.2

lc_speed_table = speed_tables.esaacci_landcover

In [6]:
# Generate pre-intervention friction surface
if not os.path.exists(pre_friction_file):
    pre_roads = major_roads.copy()
    if pre_roads.crs != in_lc.crs:
        pre_roads = pre_roads.to_crs(in_lc.crs)
    pre_friction = ma.generate_roads_lc_friction(in_lc, pre_roads, lc_travel_table=lc_speed_table, 
                             out_file=pre_friction_file, resolution=in_lc.res[0])

pre_friction = rasterio.open(pre_friction_file)

In [7]:
# Generate post-intervention friction surface
if not os.path.exists(post_friction_file):
    post_roads = major_roads.copy()  
    if post_roads.crs != in_lc.crs:
        post_roads = post_roads.to_crs(in_lc.crs)
    post_friction = ma.generate_roads_lc_friction(in_lc, post_roads, lc_travel_table=lc_speed_table, 
                              out_file=post_friction_file, resolution=in_lc.res[0], speed_col='new_speed')
    
post_friction = rasterio.open(post_friction_file)

c:\WBG\Anaconda3\envs\gostnets\lib\site-packages\rasterio\features.py:336: ShapeSkipWarning: Invalid or empty shape None at index 49450 will not be rasterized.
  warnings.warn(
c:\WBG\Anaconda3\envs\gostnets\lib\site-packages\rasterio\features.py:336: ShapeSkipWarning: Invalid or empty shape None at index 49451 will not be rasterized.
  warnings.warn(
c:\WBG\Anaconda3\envs\gostnets\lib\site-packages\rasterio\features.py:336: ShapeSkipWarning: Invalid or empty shape None at index 49452 will not be rasterized.
  warnings.warn(
c:\WBG\Anaconda3\envs\gostnets\lib\site-packages\rasterio\features.py:336: ShapeSkipWarning: Invalid or empty shape None at index 49646 will not be rasterized.
  warnings.warn(
C:\WBG\Work\Code\GOSTnetsraster\src\GOSTnetsraster\market_access.py:177: RuntimeWarning: divide by zero encountered in divide
  res = 1/((res * 1000)/60) # km/h --> m/min --> minutes/m


## Calculate travel time to ports

In [18]:
in_pop = rasterio.open(pop_file.replace(".tif", "_proj.tif"))

# Calculate pre-intervention, population-weighted travel times summarized at admin 2
frictionD = pre_friction.read()[0,:,:]
frictionD = frictionD * pre_friction.res[0]
mcp = graph.MCP_Geometric(frictionD)

ports = gpd.read_file(port_file).to_crs(m_crs)
ports = ports.to_crs(pre_friction.crs)

pre_tt_ports = ma.summarize_travel_time_populations(in_pop, pre_friction, ports, mcp, adm2, 
                                                    out_tt_file=os.path.join(results_folder, "PRE_travel_time_to_ports.tif"))
pre_tt_ports.to_file(os.path.join(results_folder, "PRE_ADM2_tt_ports.gpkg"), driver="GPKG")
pd.DataFrame(pre_tt_ports.drop(["geometry"], axis=1)).to_csv(os.path.join(results_folder, "PRE_ADM2_tt_ports.csv"))

# Run analysis using post_treatment friction surface
frictionD = post_friction.read()[0,:,:]
frictionD = frictionD * post_friction.res[0]
mcp = graph.MCP_Geometric(frictionD)

post_tt_ports = ma.summarize_travel_time_populations(in_pop, post_friction, ports, mcp, adm2, 
                                                    out_tt_file=os.path.join(results_folder, "POST_travel_time_to_ports.tif"))
post_tt_ports.to_file(os.path.join(results_folder, "POST_ADM2_tt_ports.gpkg"), driver="GPKG")
pd.DataFrame(post_tt_ports.drop(["geometry"], axis=1)).to_csv(os.path.join(results_folder, "POST_ADM2_tt_ports.csv"))

# Calculate difference in travel times and percentage change
tt_diff = post_tt_ports.copy()

for col in ['total_pop', 'pop_30', 'pop_60', 'pop_120', 'pop_180', 'pop_240', 'tt_pop_w']:
    tt_diff[col] = pre_tt_ports[col] - post_tt_ports[col]
tt_diff.to_file(os.path.join(results_folder, "TT_diff_ADM2_tt_ports.gpkg"), driver="GPKG")
pd.DataFrame(tt_diff.drop(["geometry"], axis=1)).to_csv(os.path.join(results_folder, "TT_diff_ADM2_tt_ports.csv"))

## Calculate TT to border crossings

In [19]:
in_pop = rasterio.open(pop_file.replace(".tif", "_proj.tif"))

# Calculate pre-intervention, population-weighted travel times summarized at admin 2
frictionD = pre_friction.read()[0,:,:]
frictionD = frictionD * pre_friction.res[0]
mcp = graph.MCP_Geometric(frictionD)

borders = gpd.read_file(border_crossings_file).to_crs(m_crs)
borders = borders.to_crs(pre_friction.crs)

pre_tt_borders = ma.summarize_travel_time_populations(in_pop, pre_friction, borders, mcp, adm2, 
                                                    out_tt_file=os.path.join(results_folder, "PRE_travel_time_to_borders.tif"))
pre_tt_borders.to_file(os.path.join(results_folder, "PRE_ADM2_tt_borders.gpkg"), driver="GPKG")
pd.DataFrame(pre_tt_borders.drop(["geometry"], axis=1)).to_csv(os.path.join(results_folder, "PRE_ADM2_tt_borders.csv"))

# Run analysis using post_treatment friction surface

frictionD = post_friction.read()[0,:,:]
frictionD = frictionD * post_friction.res[0]
mcp = graph.MCP_Geometric(frictionD)

post_tt_borders = ma.summarize_travel_time_populations(in_pop, post_friction, borders, mcp, adm2, 
                                                    out_tt_file=os.path.join(results_folder, "POST_travel_time_to_borders.tif"))
post_tt_borders.to_file(os.path.join(results_folder, "POST_ADM2_tt_borders.gpkg"), driver="GPKG")
pd.DataFrame(post_tt_borders.drop(["geometry"], axis=1)).to_csv(os.path.join(results_folder, "POST_ADM2_tt_borders.csv"))

# Calculate difference in travel times and percentage change
tt_diff = post_tt_borders.copy()

for col in ['total_pop', 'pop_30', 'pop_60', 'pop_120', 'pop_180', 'pop_240', 'tt_pop_w']:
    tt_diff[col] = pre_tt_borders[col] - post_tt_borders[col]

tt_diff.to_file(os.path.join(results_folder, "TT_diff_ADM2_tt_borders.gpkg"), driver="GPKG")
pd.DataFrame(tt_diff.drop(["geometry"], axis=1)).to_csv(os.path.join(results_folder, "TT_diff_ADM2_tt_borders.csv"))


In [13]:
pre_tt_borders.head()

,ISO_A3,ISO_A2,WB_A3,WB_REGION,WB_STATUS,NAM_0,NAM_1,ADM1CD_c,GEOM_SRCE,NAM_2,ADM2CD_c,geometry,total_pop,pop_30,pop_60,pop_120,pop_180,pop_240,tt_pop_w
4298,GEO,GE,GEO,ECA,Member State,Georgia,Abkhazeti (Abkhazia) Autonomous Republic,GEO001,UN SALB,Gagra,GEO001001,"POLYGON ((3.32e+06 5.18e+06, 3.32e+06 5.18e+06...",3073.968506,0.0,0.0,0.0,0.0,0.00000,537.640430
4299,GEO,GE,GEO,ECA,Member State,Georgia,Abkhazeti (Abkhazia) Autonomous Republic,GEO001,UN SALB,Gali,GEO001002,"MULTIPOLYGON (((3.46e+06 5.1e+06, 3.46e+06 5.1...",2531.226074,0.0,0.0,0.0,0.0,217.28714,260.490092
4300,GEO,GE,GEO,ECA,Member State,Georgia,Abkhazeti (Abkhazia) Autonomous Republic,GEO001,UN SALB,Gudauta,GEO001003,"POLYGON ((3.33e+06 5.14e+06, 3.33e+06 5.14e+06...",3074.738770,0.0,0.0,0.0,0.0,0.00000,465.049833
4301,GEO,GE,GEO,ECA,Member State,Georgia,Abkhazeti (Abkhazia) Autonomous Republic,GEO001,UN SALB,Gulripshi,GEO001004,"POLYGON ((3.47e+06 5.14e+06, 3.47e+06 5.14e+06...",1427.125732,0.0,0.0,0.0,0.0,0.00000,387.992443
4302,GEO,GE,GEO,ECA,Member State,Georgia,Abkhazeti (Abkhazia) Autonomous Republic,GEO001,UN SALB,Ochamchire,GEO001005,"POLYGON ((3.46e+06 5.1e+06, 3.46e+06 5.1e+06, ...",3243.102051,0.0,0.0,0.0,0.0,0.00000,338.033034


In [ ]:
# Calculate difference in travel times and percentage change
tt_diff = post_tt_borders.copy()

for col in ['total_pop', 'pop_30', 'pop_60', 'pop_120', 'pop_180', 'pop_240', 'tt_pop_w']:
    tt_diff[col] = post_tt_borders[col] - pre_tt_borders[col]

tt_diff.to_file(os.path.join(results_folder, "TT_diff_ADM2_tt_borders.gpkg"), driver="GPKG")
pd.DataFrame(tt_diff.drop(["geometry"], axis=1)).to_csv(os.path.join(results_folder, "TT_diff_ADM2_tt_borders.csv"))


In [ ]:
## TODO: Generate maps showing changes in travel time

# Write summary to readme

In [ ]:
# Write a README.md file summarizing the work so far
readme_file = "README.md"
with open(readme_file, 'w') as f:
    f.write("# GEO Road Improvements Market Access Analysis\n")
    f.write("This analysis evaluates the impact of road improvements in Georgia on travel time to ports and border crossings. \
            It compares pre- and post-intervention travel times using friction surfaces and summarizes the results at the administrative level 2 (ADM2) regions.\n \
            ")
    f.write("\n")
    f.write("Improved roads were identified from OSM data and digitized where necessary; roads were attributed with improvement status from a map provided by the project team. \
            Friction surfaces were generated using ESA CCI landcover data and road speed information. \
            Travel times to the nearest ports and border crossings were calculated using the MCP algorithm, weighted by population from WorldPop data.\n")

    f.write("![Road status and destinations map](maps/GEO_road_status_destinations.png)\n")

    f.write("## Calculating Travel Time and Improvements\n")
    f.write("Travel times to ports and border crossings were calculated using both pre- and post-intervention friction surfaces. \
            __All roads of any status were included in the calculations__ and all speeds were increased by 20% to account for improved conditions. \
            The results were summarized at the ADM2 level, providing insights into how road improvements have affected accessibility in different regions.\n")
    f.write("![Travel time to ports map](maps/GEO_tt_ports.png)\n")
    f.write("![Change in travel time to ports at ADM2 level](maps/GEO_tt_ports_ADM2_change.png)\n")

    
